# ConversionCentral Managed Profiling
Run this notebook from a Databricks Repo so backend deployments control profiling logic.

In [ ]:
# Collect parameters passed by the FastAPI backend
dbutils.widgets.text("table_group_id", "")
dbutils.widgets.text("profile_run_id", "")
dbutils.widgets.text("data_quality_schema", "")
dbutils.widgets.text("payload_path", "")
dbutils.widgets.text("callback_url", "")
dbutils.widgets.text("callback_token", "")

from datetime import datetime
import json
import requests
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
table_group_id = dbutils.widgets.get("table_group_id")
profile_run_id = dbutils.widgets.get("profile_run_id")
schema_name = dbutils.widgets.get("data_quality_schema")
payload_path = dbutils.widgets.get("payload_path")
callback_url = dbutils.widgets.get("callback_url")
callback_token = dbutils.widgets.get("callback_token")
if not table_group_id or not profile_run_id:
    raise ValueError("Required widgets missing: table_group_id/profile_run_id")

In [ ]:
# TODO: replace this stub with actual profiling implementation
# Example: load table metadata using schema widgets, run aggregations, build anomaly list
row_count = 0
anomalies = []
generated_at = datetime.utcnow().isoformat() + "Z"
results = {
    "table_group_id": table_group_id,
    "profile_run_id": profile_run_id,
    "status": "completed",
    "row_count": row_count,
    "anomaly_count": len(anomalies),
    "anomalies": anomalies,
    "generated_at": generated_at
}

In [ ]:
# Persist payload and call back into the API
if payload_path:
    dbutils.fs.put(payload_path, json.dumps(results), overwrite=True)

if callback_url:
    response = requests.post(
        callback_url,
        headers={"Authorization": f"Bearer {callback_token}", "Content-Type": "application/json"},
        json={
            "status": results["status"],
            "row_count": results["row_count"],
            "anomaly_count": results["anomaly_count"],
            "anomalies": results["anomalies"],
        },
        timeout=30,
    )
    response.raise_for_status()
results